<a href="https://colab.research.google.com/github/simulate111/Textual-Data-Analysis-25/blob/main/project_TDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> TDA course project </h1>

Name: MRA

Pair: If you did this project as pair work, name the other student here, leave empty otherwise. If you work in pair, <b>both</b> hand out the same project report in Moodle.


<h1> Step 1: Load the data with LLM judgements </h1>

In [1]:
#work here


In [2]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.5 MB/s eta 0:00:00


In [3]:
import os
import gzip
import json
import pandas as pd
from datasets import Dataset

# Step 1: Download the file
url = "http://dl.turkunlp.org/tda-course-2025/tda25-responses.jsonl.gz"
filename = "tda25-responses.jsonl.gz"

os.system(f"wget {url} -O {filename}")

# Step 2: Extract the JSONL file
extracted_filename = "tda25-responses.jsonl"

with gzip.open(filename, 'rt', encoding='utf-8') as f_in:
    data = [json.loads(line) for line in f_in]

# Step 3: Convert to Pandas DataFrame
df = pd.DataFrame(data)

# Step 4: Convert to Hugging Face Dataset
hf_dataset = Dataset.from_pandas(df)

# Verify the dataset
print(hf_dataset)


Dataset({
    features: ['document', 'response'],
    num_rows: 2515
})


In [4]:
print(hf_dataset[0])  # Print the first sample


{'document': 'Peeling an onion seems like an trivial task, but if you’ve never peeled an onion before, it can be quite intimidating. Don’t worry – it is pretty easy to peel an onion.\nYou can now learn how to peel an onion by following these illustrated step-by-step instructions.\nStep #1: Put the whole onion on the cutting board\nStep 2: Cut off one end of the onion with a knife, as shown on the picture below:\nHere’s a picture of the onion with that end already cut off. The end of the onion is laying on the right side of the onion on the cutting board.\nStep 3: Cut off another end of the onion with a knife, as show on the picture below:\nAfter both ends of the onion have been cut off, the onion is ready to be peeled. Here’s the picture of the onion without its ends:\nStep 4: Start peeling! Make a cut under the peel, and pull on the peel so it separates from the onion. Look at the picture: knife under the peel, thumb on top of the peel. Grab the peel and pull.\nStep 5: Keep peeling in

In [5]:
from transformers import AutoTokenizer

# Load the pre-trained GPT-2 tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Set the EOS token as the padding token
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the 'document' column
def tokenize_function(examples):
    return tokenizer(examples['document'], truncation=True, padding="max_length", max_length=512)

# Apply the tokenizer to the dataset
tokenized_datasets = hf_dataset.map(tokenize_function, batched=True)

# Check the tokenized dataset
print(tokenized_datasets)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/2515 [00:00<?, ? examples/s]

Dataset({
    features: ['document', 'response', 'input_ids', 'attention_mask'],
    num_rows: 2515
})


In [6]:
# Prepare the dataset to be used in the fine-tuning
def format_for_finetuning(examples):
    return {
        'input_ids': examples['input_ids'],
        'attention_mask': examples['attention_mask'],
        'labels': examples['input_ids']  # We'll use the same input for labels for a causal language modeling task
    }

# Apply the formatting function
final_dataset = tokenized_datasets.map(format_for_finetuning, batched=True)

# Check the formatted dataset
print(final_dataset)


Map:   0%|          | 0/2515 [00:00<?, ? examples/s]

Dataset({
    features: ['document', 'response', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 2515
})


<h1> Step 2: Classifier training and evaluation </h1>


*   Which target did you choose?
*   Label distribution and majority baseline
*   Classifier performance
*   Manual inspection of the classifier output, what kinds of mistakes it makes?
*   What is the composition of the data we gave you? What does it mean for your results?
*   Concusions




In [9]:
#work here

In [7]:
from transformers import Trainer, TrainingArguments, GPT2LMHeadModel

# Load pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    report_to="none",  # Disable WandB
)


# Initialize Trainer
trainer = Trainer(
    model=model,                       # The pre-trained model
    args=training_args,                # Training arguments
    train_dataset=final_dataset,       # The training dataset
    eval_dataset=final_dataset         # The evaluation dataset
)

# Train the model
trainer.train()


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,2.836400,2.350806
2,2.444400,2.241227
3,2.280300,2.203393


TrainOutput(global_step=3774, training_loss=2.479557213482556, metrics={'train_runtime': 1660.0005, 'train_samples_per_second': 4.545, 'train_steps_per_second': 2.273, 'total_flos': 1971448381440000.0, 'train_loss': 2.479557213482556, 'epoch': 3.0})

In [8]:
# Evaluate the model
trainer.evaluate()


{'eval_loss': 2.203392505645752,
 'eval_runtime': 112.8824,
 'eval_samples_per_second': 22.28,
 'eval_steps_per_second': 11.144,
 'epoch': 3.0}

<h1> Bonus step </h1>

(leave empty if you do not do this)

*   Prompt design
*   Build (prompt,response pairs)
*   Turn into HF Dataset and save



In [ ]:
#work here

<h1> Summary and Conclusions </h1>

* Brief TL;DR -style summary and main conclusions of your project.